In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import necessary modules
import os
import numpy as np
import csv
from tabulate import tabulate
import sys
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
alaska_voting = pd.read_csv('/content/drive/My Drive/415/datasci 415/alaska_voting.csv')
alaska_voting.shape

(30, 21)

In [ ]:
#alaska_voting['County']

In [ ]:
corgis = pd.read_csv('/content/drive/My Drive/415/datasci 415/county_demographics-corgis.csv')
corgis_alaska = corgis[corgis['State'] == 'AK']
almost_full = pd.read_csv('/content/drive/My Drive/415/datasci 415/joined_dataset_without_alaska.csv')
#print(almost_full.columns)
columns = almost_full.columns[0:27] #finding the columns we want from corgis data
corgis_alaska = corgis_alaska[corgis_alaska.columns.intersection(columns)]

In [ ]:
alaska_full = pd.merge(corgis_alaska, alaska_voting, on='County', how='left') #merge demo info with voting info

In [ ]:
print(alaska_full.shape)

(27, 47)


In [ ]:
#print(almost_full.columns) #need to match alaska dataset with current dataset

In [ ]:
for col in alaska_full.columns:
    if "Republican_Percent_" in col:
         alaska_full.drop(col, axis=1, inplace=True)
    if "Total_Votes_" in col:
        alaska_full[col] = alaska_full[col].str.replace(',', '')
        alaska_full[col] = alaska_full[col].astype(int)
        alaska_full.rename(columns={col:  col.lower()}, inplace=True)

    if "Democrat_Percent_" in col:
        alaska_full[col] = (
          alaska_full[col]
          .str.rstrip('%')  # Remove the percentage sign
          .astype(float)    # Convert to float
          / 100             # Divide by 100 to get the decimal value
)

for col in alaska_full.columns:
  if "Democrat_Percent_" in col:
    alaska_full[col] = alaska_full[col] - (1 - alaska_full[col])
    alaska_full.rename(columns={col: col.replace('Democrat_Percent', 'prop_favored_dem')}, inplace=True)


alaska_full.rename(columns={'County': 'county_names_with_casing'}, inplace=True)
alaska_full['County'] = alaska_full['county_names_with_casing'].str.upper()
alaska_full.drop('County Equivalent', axis=1, inplace=True)

In [ ]:
print(almost_full.shape)

(3109, 41)


In [ ]:
#sanity check
columns_df1 = set(almost_full.columns)
columns_df2 = set(alaska_full.columns)
print(columns_df2 - columns_df1)
print(columns_df1 - columns_df2)

set()
set()


In [ ]:
full = pd.concat([almost_full, alaska_full])
# line that should work to run it with
full = full.assign(winner_2020 = np.where(full["prop_favored_dem_2020"] > 0, "Democrat", "Republican"))
print(full.shape)
#print(full.columns)

#full[full['State'] == 'AK']

(3136, 42)


In [ ]:
full.to_csv('/content/drive/My Drive/415/datasci 415/full_dataset.csv', index=False)

In [ ]:
na_counts = full.isna().sum()
#print(na_counts)